In [1]:
import arcpy;
import os,sys;
import requests,csv;
import datetime,importlib;

print("Step 80: Add Station Temperature Data");

import swc_resources;
importlib.reload(swc_resources);
rez = swc_resources.rez();

start_date = '1990-01-01';
end_date   = '2020-12-31';

stations = rez['results']  + os.sep + 'SWC_Station_Universe';
stations_cnt = arcpy.GetCount_management(stations)[0];
print("  Initial SWC Station Universe Count: " + str(stations_cnt));


Step 80: Add Station Temperature Data
  Initial SWC Station Universe Count: 5144


### 80.010: Create fresh SWC_Station_TempMinMax table

In [2]:
%%time

output80010 = rez['results'] + os.sep + 'SWC_Station_TempMinMax';

print("  Creating new station tempminmax table");
if arcpy.Exists(output80010):
    arcpy.Delete_management(output80010);
  
arcpy.CreateTable_management(
     out_path      = os.path.dirname(output80010)
    ,out_name      = os.path.basename(output80010)
);

arcpy.management.AddFields(
     in_table          = output80010
    ,field_description = [
         ['StationId'    ,'TEXT'  ,'Station ID'    ,14 ]
        ,['StationSource','TEXT'  ,'Station Source',255]
        ,['Year'         ,'TEXT'  ,'Year'          ,4  ]
        ,['Month'        ,'TEXT'  ,'Month'         ,2  ]
        ,['Day'          ,'TEXT'  ,'Day'           ,2  ]
        ,['TMax'         ,'DOUBLE','Maximum Temperature']
        ,['TMin'         ,'DOUBLE','Minimum Temperature']
        ,['TSource'      ,'TEXT'  ,'Temperature Source',255]
    ]
);

print("  Adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Year'
    ,index_name = 'Year_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Month'
    ,index_name = 'Month_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80010
    ,fields     = 'Day'
    ,index_name = 'Day_IDX'
);

print("  Complete.");
del output80010;


  Creating new station tempminmax table
  Adding indexes
  Complete.
Wall time: 5.67 s


### 80.020: Create fresh SWC_Station_TempStats working table

In [4]:
%%time

output80020 = rez['results'] + os.sep + 'SWC_Station_TempStats';

print("  Creating new station tempstat working table");
if arcpy.Exists(output80020):
    arcpy.Delete_management(output80020);
  
arcpy.CreateTable_management(
     out_path      = os.path.dirname(output80020)
    ,out_name      = os.path.basename(output80020)
);

arcpy.management.AddFields(
     in_table          = output80020
    ,field_description = [
         ['StationId'             ,'TEXT'  ,'Station ID'                  ,14 ]
        ,['StationSource'         ,'TEXT'  ,'Station Source'              ,255]
        ,['RecordCount'           ,'LONG'  ,'Record Count'                    ]
        ,['YearsWithData'         ,'LONG'  ,'Years with Data'                 ]
        ,['QualifyingYears'       ,'LONG'  ,'Qualifying Years with Data'      ]
        ,['AverageDaysPerQualYear','DOUBLE','Average Days per Qualifying Year']
        ,['LongestQualifyingRun'  ,'LONG'  ,'Longest Qualifying Run'          ]
        ,['LongestRunStartYear'   ,'TEXT'  ,'Longest Run Start Year'      ,4  ]
        ,['LongestRunEndYear'     ,'TEXT'  ,'Longest Run End Year'        ,4  ]
        ,['LongestRunEnding2020'  ,'LONG'  ,'Longest Run Qualifying Run ending in 2020',4  ]
    ]
);

print("  Adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output80020
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80020
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

print("  Complete.");
del output80020;


  Creating new station tempstat working table
  Adding indexes
  Complete.
Wall time: 4.19 s


### 80.030: Create fresh SWC_Station_TempStats Gaps working table

In [4]:
%%time

output80030 = rez['results'] + os.sep + 'SWC_Station_TempStatsGaps';

print("  Creating new station tempstatgaps working table");
if arcpy.Exists(output80030):
    arcpy.Delete_management(output80030);
  
arcpy.CreateTable_management(
     out_path      = os.path.dirname(output80030)
    ,out_name      = os.path.basename(output80030)
);

arcpy.management.AddFields(
     in_table          = output80030
    ,field_description = [
         ['StationId'    ,'TEXT'  ,'Station ID'    ,14 ]
        ,['StationSource','TEXT'  ,'Station Source',255]
        ,['Year'         ,'TEXT'  ,'Year'          ,4  ]
        ,['StartMonth'   ,'TEXT'  ,'Start Month'   ,2  ]
        ,['StartDay'     ,'TEXT'  ,'Start Day'     ,2  ]
        ,['EndMonth'     ,'TEXT'  ,'End Month'     ,2  ]
        ,['EndDay'       ,'TEXT'  ,'End Day'       ,2  ]
    ]
);

print("  Adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output80030
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80030
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output80030
    ,fields     = 'Year'
    ,index_name = 'Year_IDX'
);

print("  Complete.");
del output80030;


  Creating new station tempstatday working table
  Adding indexes
  Complete.
Wall time: 4.1 s


### 80.040: Download NCEI station temperature data to CSV files.

Note the following step may require 3 to 5 hours to complete depending on the speed of your internet connection and the relative busy-ness of the ncei servers.  Executing in the off hours may result in the lower end of the estimate.

In [10]:
%%time

input80040  = rez['results'] + os.sep + 'SWC_Station_Universe';
output80040 = rez['files']   + os.sep + 'Station_ncei_data.csv';
stats80040  = rez['files']   + os.sep + 'Station_ncei_stats.csv';

fldsin = [
     'StationId'
    ,'StationSource'
];

print("  Downloading station data from ncei between " + str(start_date) + " and " + str(end_date) + ".");
station_count = 0;
with arcpy.da.SearchCursor(
     in_table    = input80040
    ,field_names = fldsin
) as incur:
    
    with open(output80040,'w') as outfile \
        ,open(stats80040,'w')  as statfile:
    
        outfile.write('"STATION","DATE","TMAX","TMIN","SOURCE"\n');
        statfile.write('"STATION","RECORD_COUNT"\n');
        
        for row in incur:
            
            stationID = row[0];
            stationSource = row[1];
            
            #print("   StationID: " + stationID);
            if stationSource == "COOP":
                payload = {
                     'dataset'          : 'daily-summaries'
                    ,'dataTypes'        : 'TMAX,TMIN'
                    ,'stations'         : stationID
                    ,'startDate'        : start_date
                    ,'endDate'          : end_date
                    ,'includeAttributes': 'false'
                    ,'format'           : 'csv'
                    ,'units'            : 'standard'
                }
            
            elif stationSource == "ISD":
                payload = {
                     'dataset'          : 'global-summary-of-the-day'
                    ,'dataTypes'        : 'MAX,MIN'
                    ,'stations'         : stationID
                    ,'startDate'        : start_date
                    ,'endDate'          : end_date
                    ,'includeAttributes': 'false'
                    ,'format'           : 'csv'
                }
                
            r = requests.get(
                'https://www.ncei.noaa.gov/access/services/data/v1'
                ,params = payload        
            );
            
            rec_count = 0;
            text = (line.decode('utf-8') for line in r.iter_lines());
            rows = csv.reader(text,delimiter=",",quotechar='"');
            next(rows);
            
            for line in rows:
                
                stationID   = line[0];
                readingDate = line[1];
                readingMax  = line[2];
                readingMin  = line[3];
                
                if readingMax is not None and readingMin is not None \
                and readingMax != "" and readingMin != "":
                    
                    outfile.write('"' + stationID + '","' + readingDate + '","' + readingMax + '","' + readingMin + '","' + stationSource + '"\n');
                    rec_count += 1;
                
            statfile.write('"' + stationID + '","' + stationSource + '",' + str(rec_count) + "\n");
            station_count += 1;
            
print("  Downloaded " + str(station_count) + " stations worth of temperature data.");
       

  Loop through station universe downloading ncei data.
   StationID: USC00010008
   StationID: USC00010063
   StationID: USC00010140
   StationID: USC00010252
   StationID: USC00010369
   StationID: USC00010390
   StationID: USC00010402
   StationID: USC00010425
   StationID: USC00010748
   StationID: USC00010957
   StationID: USC00011099
   StationID: USC00012124
   StationID: USC00012172
   StationID: USC00012377
   StationID: USC00012675
   StationID: USC00013043
   StationID: USC00013519
   StationID: USC00013620
   StationID: USC00013645
   StationID: USC00014193
   StationID: USC00014209
   StationID: USC00015397
   StationID: USC00015553
   StationID: USC00018178
   StationID: USC00018209
   StationID: USC00018385
   StationID: USC00018517
   StationID: USC00018670
   StationID: USC00018673
   StationID: USC00020080
   StationID: USC00020487
   StationID: USC00020808
   StationID: USC00021574
   StationID: USC00021749
   StationID: USC00021870
   StationID: USC00022329
   Statio

   StationID: USC00090044
   StationID: USC00091619
   StationID: USC00092361
   StationID: USC00092485
   StationID: USC00092844
   StationID: USC00093028
   StationID: USC00093312
   StationID: USC00093460
   StationID: USC00094204
   StationID: USC00094623
   StationID: USC00094671
   StationID: USC00094941
   StationID: USC00095165
   StationID: USC00095314
   StationID: USC00096879
   StationID: USC00097605
   StationID: USC00098657
   StationID: USC00098974
   StationID: USC00099291
   StationID: USC00099486
   StationID: USC00101018
   StationID: USC00101079
   StationID: USC00101956
   StationID: USC00102159
   StationID: USC00102575
   StationID: USC00102707
   StationID: USC00102845
   StationID: USC00102875
   StationID: USC00103143
   StationID: USC00103677
   StationID: USC00103732
   StationID: USC00103773
   StationID: USC00103811
   StationID: USC00104230
   StationID: USC00104456
   StationID: USC00104598
   StationID: USC00104845
   StationID: USC00105177
   StationID

   StationID: USC00188065
   StationID: USC00200089
   StationID: USC00200230
   StationID: USC00200662
   StationID: USC00200766
   StationID: USC00201088
   StationID: USC00201361
   StationID: USC00201486
   StationID: USC00201680
   StationID: USC00201780
   StationID: USC00202094
   StationID: USW00014824
   StationID: USW00014828
   StationID: USC00203188
   StationID: USC00203295
   StationID: USC00203391
   StationID: USC00203516
   StationID: USC00203585
   StationID: USC00203947
   StationID: USC00204090
   StationID: USC00204104
   StationID: USC00204155
   StationID: USC00204320
   StationID: USC00204328
   StationID: USC00204502
   StationID: USC00205073
   StationID: USW00094850
   StationID: USC00205567
   StationID: USC00205816
   StationID: USC00206215
   StationID: USC00206300
   StationID: USC00207407
   StationID: USC00207812
   StationID: USC00207828
   StationID: USC00208246
   StationID: USC00208293
   StationID: USC00208417
   StationID: USC00208443
   StationID

   StationID: USC00265846
   StationID: USC00265880
   StationID: USC00267192
   StationID: USC00267369
   StationID: USC00267612
   StationID: USC00267750
   StationID: USC00268822
   StationID: USC00268838
   StationID: USC00270998
   StationID: USC00272174
   StationID: USC00272842
   StationID: USC00273182
   StationID: USC00273850
   StationID: USC00274218
   StationID: USC00274806
   StationID: USC00275780
   StationID: USC00275868
   StationID: USC00276234
   StationID: USC00276818
   StationID: USC00276856
   StationID: USC00278885
   StationID: USC00281351
   StationID: USC00290041
   StationID: USC00290407
   StationID: USC00290417
   StationID: USC00290600
   StationID: USC00290640
   StationID: USC00290818
   StationID: USC00291120
   StationID: USC00291286
   StationID: USC00291440
   StationID: USC00291445
   StationID: USC00291469
   StationID: USC00291963
   StationID: USC00291982
   StationID: USC00292030
   StationID: USC00292100
   StationID: USC00292241
   StationID

   StationID: USC00358263
   StationID: USC00358504
   StationID: USC00358512
   StationID: USC00358717
   StationID: USC00358790
   StationID: USC00358884
   StationID: USC00359213
   StationID: USC00359581
   StationID: USC00360147
   StationID: USC00360560
   StationID: USC00360725
   StationID: USC00360785
   StationID: USC00360821
   StationID: USC00361139
   StationID: USC00361726
   StationID: USC00361961
   StationID: USC00362245
   StationID: USC00362323
   StationID: USC00362721
   StationID: USC00363018
   StationID: USC00363028
   StationID: USC00363321
   StationID: USC00364001
   StationID: USC00364325
   StationID: USC00364432
   StationID: USC00364763
   StationID: USC00364778
   StationID: USC00365731
   StationID: USC00365790
   StationID: USC00366111
   StationID: USC00366233
   StationID: USC00366792
   StationID: USC00366916
   StationID: USC00367186
   StationID: USC00367229
   StationID: USC00367931
   StationID: USC00367938
   StationID: USC00368026
   StationID

...

   StationID: 72549304962
   StationID: 72549399999
   StationID: 72549404966
   StationID: 72549499999
   StationID: 72549504975
   StationID: 72549599999
   StationID: 72549604976
   StationID: 72549699999
   StationID: 72549704908
   StationID: 72549799999
   StationID: 72549894998
   StationID: 72549994991
   StationID: 72550014942
   StationID: 72551014939
   StationID: 72551214989
   StationID: 72551304901
   StationID: 72551403757
   StationID: 72551594947
   StationID: 72551599999
   StationID: 72552014935
   StationID: 72552494958
   StationID: 72552499999
   StationID: 72552594949
   StationID: 72552599999
   StationID: 72552614905
   StationID: 72552699999
   StationID: 72552794978
   StationID: 72552799999
   StationID: 72553394957
   StationID: 72554014949
   StationID: 72554104993
   StationID: 72555594946
   StationID: 72555599999
   StationID: 72555694975
   StationID: 72555699999
   StationID: 72556014941
   StationID: 72556404924
   StationID: 72556499999
   StationID

   StationID: 72655614992
   StationID: 72655714910
   StationID: 72655804914
   StationID: 72655899999
   StationID: 72655994976
   StationID: 72655999999
   StationID: 72656024025
   StationID: 72656099999
   StationID: 72656194997
   StationID: 72656204943
   StationID: 72656394969
   StationID: 72656399999
   StationID: 72656404967
   StationID: 72656499999
   StationID: 72656504948
   StationID: 72656599999
   StationID: 72656604965
   StationID: 72656699999
   StationID: 72656704980
   StationID: 72656799999
   StationID: 72656804961
   StationID: 72656899999
   StationID: 72656904933
   StationID: 72656999999
   StationID: 72657294966
   StationID: 72657494985
   StationID: 72657499999
   StationID: 72657594960
   StationID: 72657599999
   StationID: 72657614928
   StationID: 72657794974
   StationID: 72657799999
   StationID: 72657804944
   StationID: 72657899999
   StationID: 72657994963
   StationID: 72657999999
   StationID: 72658014922
   StationID: 72658304941
   StationID

   StationID: 72798594276
   StationID: 72798599999
   StationID: 74000154793
   StationID: 74000203042
   StationID: 74003024103
   StationID: 74003099999
   StationID: 74003503145
   StationID: 74200699999
   StationID: 74201099999
   StationID: 74206024207
   StationID: 74207024201
   StationID: 74207703763
   StationID: 74207864773
   StationID: 74207963876
   StationID: 74230024037
   StationID: 74251353937
   StationID: 74331253925
   StationID: 74370014715
   StationID: 74370099999
   StationID: 74392014611
   StationID: 74394514710
   StationID: 74394654754
   StationID: 74394699999
   StationID: 74410414763
   StationID: 74421494852
   StationID: 74421499999
   StationID: 74455094982
   StationID: 74465253897
   StationID: 74465363814
   StationID: 74465504808
   StationID: 74465599999
   StationID: 74465653891
   StationID: 74465753887
   StationID: 74465853889
   StationID: 74465953822
   StationID: 74466199999
   StationID: 74466263817
   StationID: 74466353950
   StationID

   StationID: 99735399999
   StationID: 99735499999
   StationID: 99735599999
   StationID: 99735699999
   StationID: 99735799999
   StationID: 99735899999
   StationID: 99735999999
   StationID: 99736099999
   StationID: 99736199999
   StationID: 99736499999
   StationID: 99736699999
   StationID: 99736899999
   StationID: 99736999999
   StationID: 99737099999
   StationID: 99737199999
   StationID: 99737299999
   StationID: 99738099999
   StationID: 99738199999
   StationID: 99738299999
   StationID: 99738399999
   StationID: 99738499999
   StationID: 99738699999
   StationID: 99738999999
   StationID: 99768799999
   StationID: 99768899999
   StationID: 99768999999
   StationID: 99769199999
   StationID: 99769299999
   StationID: 99769399999
   StationID: 99769499999
   StationID: 99769599999
   StationID: 99769699999
   StationID: 99769799999
   StationID: 99769899999
   StationID: 99770199999
   StationID: 99770299999
   StationID: 99770399999
   StationID: 99770499999
   StationID

   StationID: 99999953968
   StationID: 99999953974
   StationID: 99999954794
   StationID: 99999954795
   StationID: 99999954796
   StationID: 99999954797
   StationID: 99999954808
   StationID: 99999954810
   StationID: 99999954811
   StationID: 99999954854
   StationID: 99999954902
   StationID: 99999954903
   StationID: 99999954932
   StationID: 99999954933
   StationID: 99999954937
   StationID: 99999963826
   StationID: 99999963828
   StationID: 99999963829
   StationID: 99999963831
   StationID: 99999963838
   StationID: 99999963849
   StationID: 99999963850
   StationID: 99999963855
   StationID: 99999963856
   StationID: 99999963857
   StationID: 99999963858
   StationID: 99999963862
   StationID: 99999963867
   StationID: 99999963868
   StationID: 99999963869
   StationID: 99999963871
   StationID: 99999963891
   StationID: 99999963892
   StationID: 99999963893
   StationID: 99999963894
   StationID: 99999963895
   StationID: 99999963896
   StationID: 99999963897
   StationID

### 80.050: Load NCEI station temperature data into geodatabase.

In [ ]:
%%time

input80050   = rez['files']   + os.sep + 'Station_ncei_data.csv';
statin80050  = rez['files']   + os.sep + 'Station_ncei_stats.csv';

output80050  = rez['results'] + os.sep + 'SWC_Station_TempMinMax';
statout80050 = rez['results'] + os.sep + 'SWC_Station_TempStats';

arcpy.TruncateTable_management(output80050);
arcpy.TruncateTable_management(statout80050);

statflds = [
     'StationID'
    ,'StationSource'
    ,'RecordCount'
];

print("  Loading station stats.");
with open(statin80050,encoding="utf8") as csv_file:
    
    with arcpy.da.InsertCursor(
         in_table    = statout80050
        ,field_names = statflds
    ) as outcurs:
    
        rows = csv.reader(csv_file,delimiter=",",quotechar='"');
        next(rows);
            
        for row in rows:
            outcurs.insertRow((
                 row[0]
                ,""
                ,row[1]
            ));
            
outflds = [
     'StationID'
    ,'StationSource'
    ,'Year'
    ,'Month'
    ,'Day'
    ,'TMax'
    ,'TMin'
    ,'TSource'
];
            
print("  Loading station data.");
with open(input80050,encoding="utf8") as csv_file:
    
    with arcpy.da.InsertCursor(
         in_table    = output80050
        ,field_names = outflds
    ) as outcurs:
    
        rows = csv.reader(csv_file,delimiter=",",quotechar='"');
        next(rows);
            
        for row in rows:
            dt = datetime.datetime.strptime(row[1],"%Y-%m-%d");
            
            outcurs.insertRow((
                 row[0]
                ,row[4]
                ,dt.strftime('%Y')
                ,dt.strftime('%m')
                ,dt.strftime('%d')
                ,float(row[2])
                ,float(row[3])
                ,'ncei.noaa.gov'
            ));
            
print("  Data load complete.");
del input80050,statin80050,output80050,statout80050


  Loading station stats.
  Loading station data.


### 80.060 Generate stats and gaps against ncei station data

In [ ]:
input80060   = rez['results'] + os.sep + 'SWC_Station_TempMinMax';
stats80060   = rez['results'] + os.sep + 'SWC_Station_TempStats';

gapout80060  = rez['results'] + os.sep + 'SWC_Station_TempStatsGaps';

arcpy.TruncateTable_management(dayout80050);

flds = [
     'StationId'
    ,'StationSource'
    ,'RecordCount'
];
print("  Load stations into storage array.");
stations = [];
with arcpy.da.SearchCursor(
     in_table    = stats80060
    ,field_names = flds
) as outcurs:
    
    for row in outcurs:
        stations.append({
             "StationId"    : row[0]
            ,"StationSource": row[1]
            ,"RecordCount"  : row[2]
        });
        
print("  Found " + str(len(stations)) + " stations to examine.");

inflds = [
     'StationId'
    ,'StationSource'
    ,'Year'
    ,'Month'
    ,'Day'  
];

gapflds = [
     'StationId'
    ,'StationSource'
    ,'Year'
    ,'StartMonth'
    ,'StartDay'
    ,'EndMonth'
    ,'EndDay'
];

for station in stations:
    
    with arcpy.da.SearchCursor(
         in_table    = input80060
        ,field_names = inflds
    ) as incurs:
    
        with arcpy.da.InsertCursor(
             in_table    = gapout80060
            ,field_names = gapflds
        ) as outcurs:
            
            if station["RecordCount"] > 0:
                
                last_row = None;
                for row in incurs:
                    
                    if last_row 
                    
    
    
    
with arcpy.da.UpdateCursor(
     in_table    = stats80060
    ,field_names = outflds
) as outcurs:
        
        